# Part 4: RAG with ColPali using MCP + Claude Desktop

**Important Notice**: The Elastic MCP Server (`mcp-server-elasticsearch`) is a Node.js application and cannot be executed directly within a Jupyter Notebook. This notebook serves as a comprehensive guide to installing and running the MCP Server and configuring Claude Desktop. After running the initial setup cells here, you must execute the provided commands in your local terminal to complete the setup and run the demo.

Follow the step-by-step guide below to configure the Elastic MCP Server, connect it with Claude Desktop, and perform a natural language search demo on the RVL-CDIP dataset. This demo showcases the Retrieval-Augmented Generation (RAG) approach, where the MCP Server retrieves relevant documents (Retrieval) and the LLM in Claude Desktop generates responses based on the search results (Generation).

**Objective**: Set up the Elastic MCP Server, connect it with Claude Desktop, and implement a natural language search demo for the RVL-CDIP dataset in the ColPali project.

**[EN]** Load environment variables from `elastic.env` to retrieve connection details for Elasticsearch.<br>
**[KR]** `elastic.env` 파일에서 환경 변수를 로드하여 Elasticsearch 연결 정보를 가져옵니다.

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from elastic.env
dotenv_path = 'elastic.env'
load_dotenv(dotenv_path=dotenv_path)

# Retrieve ES_URL and ES_API_KEY (mapping from ELASTIC_HOST and ELASTIC_API_KEY if needed)
ES_URL = os.getenv("ELASTIC_HOST", os.getenv("ES_URL", ""))
ES_API_KEY = os.getenv("ELASTIC_API_KEY", os.getenv("ES_API_KEY", ""))

if not ES_URL or not ES_API_KEY:
    raise ValueError(f"Please create an '{dotenv_path}' file and set ES_URL (or ELASTIC_HOST) and ES_API_KEY variables.")

print("--- Credentials Loaded ---")
print(f"ES_URL: {ES_URL}")
print(f"ES_API_KEY: {ES_API_KEY[:5]}... (partially hidden for security)")
print("\nPlease use these values in the terminal commands below.")

--- Credentials Loaded ---
ES_URL: colpali-demo:YXAtbm9ydGhlYXN0LTIuYXdzLmVsYXN0aWMtY2xvdWQuY29tOjQ0MyQ5NGQ2MWIxY2ZmZTM0MmJiYjYxZmY1M2MyNDM1ZGQzMSRlNjFjOGM5ZWE5NDQ0YTQ5YjJhODYwMzE3ZmRiMjQ0YQ==
ES_API_KEY: eGRqR... (partially hidden for security)

Please use these values in the terminal commands below.


**[EN]** Create a `package.json` file to define the MCP Server dependency. This allows for easy installation using npm.<br>
**[KR]** MCP 서버 의존성을 정의하기 위해 `package.json` 파일을 생성합니다. 이 파일을 통해 npm으로 쉽게 설치할 수 있습니다.

In [2]:
# Step 1: Create package.json for MCP Server Installation
import json

package_json = {
    "name": "colpali-mcp-demo",
    "version": "1.0.0",
    "description": "A demo project to connect Claude Desktop with Elasticsearch via MCP.",
    "dependencies": {
        "@elastic/mcp-server-elasticsearch": "latest"
    }
}

with open('package.json', 'w') as f:
    json.dump(package_json, f, indent=2)

print("package.json created successfully.")
print("Next, open your terminal and run 'npm install' to install the MCP Server.")

package.json created successfully.
Next, open your terminal and run 'npm install' to install the MCP Server.


**[EN]** The following steps must be performed in your local terminal, not in this notebook.<br>
**[KR]** 다음 단계들은 이 노트북이 아닌, 로컬 터미널에서 직접 수행해야 합니다.

### Step 2 to 4: Terminal Guide for MCP Server and Claude Desktop Setup

--- 

#### **Step 2: Install and Run MCP Server**

1.  **Open a terminal** in this project's root directory.
2.  **Install the MCP Server** package by running:
    ```bash
    npm install
    ```
3.  **Set environment variables** using the values printed in the first code cell above:
    *For macOS/Linux:*
    ```bash
    export ES_URL="<your-elasticsearch-url>"
    export ES_API_KEY="<your-api-key>"
    ```
    *For Windows (Command Prompt):*
    ```bash
    set ES_URL="<your-elasticsearch-url>"
    set ES_API_KEY="<your-api-key>"
    ```
4.  **Run the MCP Server**:
    ```bash
    npx @elastic/mcp-server-elasticsearch
    ```
    *Keep this terminal window open. The server must be running for Claude Desktop to connect.*

--- 

#### **Step 3: Configure Claude Desktop**

1.  Open the **Claude Desktop App**.
2.  Navigate to `Settings > Developer > MCP Servers` and click **'Edit Config'**.
3.  Add the following JSON configuration. Replace the placeholder values with your actual credentials from the `.env` file.
    ```json
    {
        "mcpServers": {
            "elasticsearch-mcp-server": {
                "command": "npx",
                "args": ["-y", "@elastic/mcp-server-elasticsearch"],
                "env": {
                    "ES_URL": "<your-elasticsearch-url>",
                    "ES_API_KEY": "<your-api-key>"
                }
            }
        }
    }
    ```
4. Save the configuration file. Claude Desktop should now be connected to your local MCP server.

--- 

#### **Step 4: Perform Natural Language Search**

1.  Start a new conversation in Claude Desktop.
2.  Use natural language to interact with your Elasticsearch data. For example:
    * `@elasticsearch-mcp-server list_indices`
    * `@elasticsearch-mcp-server get_mappings for index colpali-rag-demo-part2-original`
    * `@elasticsearch-mcp-server search index colpali-rag-demo-part2-original with query: { "knn": { ... } }`
3.  Observe how the MCP Server processes these commands and returns results from Elasticsearch directly into your chat.

**[EN]** Below is an overview of the tasks provided by the Elastic MCP Server for interacting with your Elasticsearch data.<br>
**[KR]** 다음은 Elastic MCP 서버가 제공하는 Elasticsearch 데이터와의 상호작용 Task 목록입니다.

### Elastic MCP Server Tasks Overview

Once the MCP Server is running and connected, you can invoke the following tasks in Claude Desktop by mentioning the server (`@elasticsearch-mcp-server`) followed by the task name and its arguments:

-   **`list_indices`**: Lists all available indices in your Elasticsearch cluster.
-   **`get_mappings`**: Retrieves the field mappings for a specified index.
-   **`search`**: Performs a search using the full Elasticsearch Query DSL. This is the most powerful task for retrieval.
-   **`get_shards`**: Gets shard information for one or more indices.